In [12]:
import os
import json
import re

In [13]:
CLI_OUTPUT_DIR = "/local/juro4948/data/egoexo4d/egoexo" # Replace with the full path to the --output_directory you pass to the cli
ANNOTATIONS_PATH = os.path.join(CLI_OUTPUT_DIR, "annotations")

# See raw annotations in this dictionary
keystep_anns = json.load(open(os.path.join(ANNOTATIONS_PATH, "keystep_train.json")))
keystep_anns_val = json.load(open(os.path.join(ANNOTATIONS_PATH, "keystep_val.json")))

anns = keystep_anns["annotations"]
print(f'Length of training items: {len(anns)}')
anns_test = keystep_anns_val["annotations"]

# Add anns_test to anns dictionary
anns.update(anns_test)
print(f'Length of training + val items: {len(anns)}')

def get_take_id_from_name(take_name):
    for take_id in anns.keys():
        if anns[take_id]['take_name'] == take_name:
            return take_id
    return None

def get_take_name_from_id(take_id):
    if take_id in anns.keys():
        return anns[take_id]['take_name']
    else:
        return None
    

Length of training items: 671
Length of training + val items: 852


In [14]:
# create symlink to each take 
# assuming that same take's aria and cam are all the in the same split 

import os
import shutil

use_aria = True # True = use aria, False = use all gopro views

n_splits = [1] #Blake: Changed to one split

# path to split bundle files
root = "/home/blde8334/research/GraVi-T-custom/data/annotations/egoexo/splits" # Blake: Changed to my direcotry
# dataset = 'egoexo'

# path to npy egoexo features
features_dir = '/home/blde8334/research/Bridge-Prompt-custom/data/egoexo/features_dir/egoexo_vit_features_splt3/combined_feat' # Blake: Changed to my directory
all_features_fn = os.listdir(features_dir)

if use_aria == True:
    out_dir = '/home/blde8334/research/GraVi-T-custom/data/features/egoexo-aria-brp'
else:
    out_dir = '/home/juro4948/gravit/GraVi-T/data/features/egoexo-omnivore-gopro'


aria_ext = '_aria.._rgb.npy'
# aria_save_ext = '_aria.npy'
gopro_ext = '_cam.._0.npy'

file_list = []
for n_split in n_splits:
    split_dir = os.path.join(out_dir, f'split{n_split}')
    # if dir already exists write over it
    if os.path.exists(split_dir):
        shutil.rmtree(split_dir)
    os.makedirs(split_dir)
    for mode in ['train', 'test']:
        mode_name = mode
        if mode_name == 'test':
            mode_name = 'val'
        os.makedirs(os.path.join(split_dir, mode_name))
        print(f'Split: {n_split} # Mode: {mode}')

        # read every take name from the split file
        txt_path = f'{mode}.split' + str(n_split) + '.bundle'
        with open(os.path.join(root, txt_path), 'r') as f:
            for line in f.readlines():
                line = line.strip('\n')
                line = line.split('.')[0]

                take_name = line 
                take_id = get_take_id_from_name(take_name)
                if take_id is None:
                    print('ID not found. Skipping')
                    continue

                # symlink the pt file
                if use_aria == True:
                    pattern = re.compile(take_id + aria_ext)
                else:
                    pattern = re.compile(take_id + gopro_ext)

                i = 0
                # find all files with matching pattern (finding all views of the same take that correspond to aria vs gopro), and save each symlink
                print("all_features_fn: ", all_features_fn)
                for fn in all_features_fn:
                    #if pattern.match(fn):
                    src = os.path.join(features_dir, fn)
                    print("Source: ", src)
                    dst = os.path.join(split_dir, mode_name, take_name + '_' + str(i) + '.npy')
                    print("Destination: ", dst)
                    i += 1

                    if os.path.exists(dst):
                        continue
                    if not os.path.exists(src):
                        print(f'File not found: {src}')
                        
                    os.symlink(src, dst)
                    print('Creating symlink')
                    

Split: 1 # Mode: train
all_features_fn:  ['iiith_cooking_120_6.npy', 'georgiatech_cooking_04_02_4.npy', 'iiith_cooking_112_4.npy', 'georgiatech_cooking_02_02_6.npy', 'iiith_cooking_132_4.npy', 'sfu_cooking_002_7.npy', 'indiana_cooking_26_4.npy', 'iiith_cooking_43_1.npy', 'upenn_0711_Cooking_6_4.npy', 'indiana_cooking_12_2.npy', 'upenn_0714_Cooking_1_3.npy', 'iiith_cooking_57_2.npy', 'sfu_cooking017_2.npy', 'indiana_cooking_03_2.npy', 'uniandes_cooking_009_2.npy', 'upenn_0714_Cooking_1_5.npy', 'minnesota_cooking_050_2.npy', 'indiana_cooking_27_2.npy', 'indiana_cooking_09_2.npy', 'georgiatech_cooking_11_01_6.npy', 'indiana_cooking_23_4.npy', 'sfu_cooking020_2.npy', 'iiith_cooking_120_2.npy', 'upenn_0702_Cooking_4_2.npy', 'indiana_cooking_20_2.npy', 'upenn_0714_Cooking_5_2.npy', 'indiana_cooking_16_3.npy', 'iiith_cooking_18_1.npy', 'minnesota_cooking_030_2.npy', 'sfu_cooking_005_6.npy', 'minnesota_cooking_074_2.npy', 'iiith_cooking_26_1.npy', 'upenn_0714_Cooking_7_2.npy']
Source:  /home/b

In [4]:
split_dirs = [os.path.join(out_dir, f'split{n_split}') for n_split in n_splits]
file_counts = [len(os.listdir(split_dir)) for split_dir in split_dirs]

file_counts

[2]